In [1]:
import pandas as pd
from sqlalchemy import create_engine

# pip install alpha_vantage
from alpha_vantage.timeseries import TimeSeries
import matplotlib.pyplot as plt

In [2]:
# Obtain free api key from alphavantage.co
api_key = "5GHY1AE5BGE4O8HT"
tickers = ["DJI", "^GSPC", "^FTSE", "^HSI", "^N225"]


In [3]:
# Be sure to update rds_connection_string with database connection info
# Alphavantage limits API usage to 5 requests per minute

rds_connection_string = "postgres:postgres@database-1.cpnwuinodiya.us-east-1.rds.amazonaws.com:5432/market_data"
engine = create_engine(f'postgresql://{rds_connection_string}')

# If index_table already exists, delete it
with engine.connect() as connection:
    result = connection.execute("DROP TABLE IF EXISTS index_table;")


In [4]:
# function to append each dataframe to SQL database

def writeData(data): 
    data.to_sql(name='index_table', con=engine, if_exists='append', index=False)

In [5]:
# Load TimeSeries object and unpack Alphavantage timeseries into dataframe
# Rename columns for easier reading and reset index value

ts = TimeSeries(key=api_key, output_format='pandas')
for ticker in tickers:
    data, metadata = ts.get_daily(symbol=ticker, outputsize='full')
    data['ticker'] = ticker
    data = data.rename(columns={"date" : "timestamp", "1. open": "open", "2. high" : "high", "3. low": "low", "4. close" : "close", "5. volume" : "volume"})
    data['change'] = data.groupby('ticker').close.pct_change()
    data = data.reset_index()
    writeData(data)    

In [6]:
df = pd.read_sql_query('select * from index_table', con=engine)

In [7]:
df.head()

,date,open,high,low,close,volume,ticker,change
0,2000-01-03,11501.8496,11522.0098,11305.6904,11357.5098,169750000.0,DJI,NaN
1,2000-01-04,11349.7500,11350.0596,10986.4502,10997.9297,178420000.0,DJI,-0.031660
2,2000-01-05,10989.3701,11215.0996,10938.6699,11122.6504,203190000.0,DJI,0.011340
3,2000-01-06,11113.3701,11313.4502,11098.4502,11253.2598,176550000.0,DJI,0.011743
4,2000-01-07,11247.0596,11528.1396,11239.9199,11522.5596,184900000.0,DJI,0.023931
